In [31]:
import cobra.test
import os
from os.path import join
from cobra.io import read_sbml_model
model = cobra.io.load_json_model(join("RehMBEL1391_sbml_L3V1.json"))
from cobra import Model, Reaction, Metabolite

# Adding D-glucose to the medium and cheching out other C-sources

The model we are using has D-fructorse as carbon source.

## Standard medium content of the model

In [32]:
model.medium

{'EX_fe2_e': 10.0,
 'EX_mg2_e': 10.0,
 'EX_pi_e': 100.0,
 'EX_cobalt2_e': 10.0,
 'EX_cl_e': 10.0,
 'EX_k_e': 10.0,
 'EX_fe3_e': 10.0,
 'EX_so4_e': 10.0,
 'EX_fru_e': 3.0,
 'EX_nh4_e': 10.0,
 'EX_na_e': 10.0,
 'EX_o2_e': 18.5,
 'EX_mobd_e': 10.0,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 100.0}

Taking a look at the fluxes

In [33]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
fe2_e,EX_fe2_e,6.333E-05,0,0.00%
fru_e,EX_fru_e,3,6,100.00%
h_e,EX_h_e,10.72,0,0.00%
nh4_e,EX_nh4_e,3.038,0,0.00%
o2_e,EX_o2_e,5.734,0,0.00%
pi_e,EX_pi_e,7.987,0,0.00%
so4_e,EX_so4_e,0.05797,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
BIOMASS_c,EX_BIOMASS_c,-0.2853,0,0.00%
co2_e,EX_co2_e,-6.151,1,100.00%


The fluxes when we change the objective:

In [34]:
model.objective = model.reactions.EX_pbhb_e
model.summary

<bound method Model.summary of <Model RehMBEL1391 at 0x1764e61c8e0>>

#### We will be trying to include D-glucose in the medium and look at the bimass and production of PHB.

With the current medium with fructose, the biomass, productivity and yeald of phb:

In [56]:
with model:
    medium = model.medium
    medium['EX_fru_e'] = 3
    print(medium)
    model.medium = medium
    
    print(model.summary())
    
    model.objective = model.reactions.EX_pbhb_e
    phb_production=model.optimize().objective_value
    solution= model.optimize()
    
    print('The current theoretical maximum biomass productivity:', solution.fluxes['EX_BIOMASS_c'], 'h')
    print('----------------')

    print('The current maximum theoretical productivity for making PHB:', phb_production , 'mmol/gDW*h')
    print('----------------')
    maximum_yield = phb_production / (-1*(model.reactions.EX_fru_e.flux))
    print('Maximum theoretical yield =', maximum_yield, 'mmol phb/mmol fructose') 

{'EX_fe2_e': 10.0, 'EX_mg2_e': 10.0, 'EX_pi_e': 100.0, 'EX_cobalt2_e': 10.0, 'EX_cl_e': 10.0, 'EX_k_e': 10.0, 'EX_fe3_e': 10.0, 'EX_so4_e': 10.0, 'EX_fru_e': 3, 'EX_nh4_e': 10.0, 'EX_na_e': 10.0, 'EX_o2_e': 18.5, 'EX_mobd_e': 10.0, 'EX_h2o_e': 1000.0, 'EX_h_e': 100.0, 'EX_glc_e': 1000.0}
Objective
1.0 EX_pbhb_e = 3.7627118644067785

Uptake
------
Metabolite  Reaction  Flux  C-Number   C-Flux
     fru_e  EX_fru_e     3         6  100.00%
      o2_e   EX_o2_e 1.068         0    0.00%

Secretion
---------
Metabolite   Reaction   Flux  C-Number   C-Flux
     co2_e   EX_co2_e -2.949         1  100.00%
     h2o_e   EX_h2o_e -6.712         0    0.00%
     phb_e  EX_pbhb_e -3.763         0    0.00%

The current theoretical maximum biomass productivity: 0.0 h
----------------
The current maximum theoretical productivity for making PHB: 3.7627118644067785 mmol/gDW*h
----------------
Maximum theoretical yield = 1.2542372881355928 mmol phb/mmol fructose


### To add D-glucose to the medium we need new reactions to make R.eutropha take up and use D-glucose as a carbon source. 

In [62]:
model.add_metabolites([
    Metabolite(
    'glc_e',
    name='D-glucose',
    compartment='e'
    )])

In [63]:
model.add_boundary(model.metabolites.get_by_id("glc_e"), type="exchange")

ValueError: Boundary reaction 'EX_glc_e' already exists.

In [64]:
model.add_metabolites([
    Metabolite(
    'glc_c',
    name='D-glucose',
    compartment='c'
    )])

In [65]:
model.add_boundary(model.metabolites.get_by_id("glc_c"), type="sink")

ValueError: Boundary reaction 'SK_glc_c' already exists.

In [39]:
#reaction1 = Reaction('Exchange_glc')

In [40]:
#reaction1.add_metabolites({
#    model.metabolites.glc_e: -1.0,
#    model.metabolites.glc_c: 1.0,
                            })
#print(reaction.build_reaction_string())

NameError: name 'reaction' is not defined

In [60]:
reaction = Reaction('PEP')

In [61]:
reaction.add_metabolites({
    model.metabolites.glc_c: -1.0,
    model.metabolites.pep_c: -1.0,
    model.metabolites.pyr_c: 1.0,
    model.metabolites.g6p_c: 1.0
                            })
print(reaction.build_reaction_string())

glc_c + pep_c --> g6p_c + pyr_c


### Adding glucose in the media and removing fructose:

In [66]:
with model:
    medium = model.medium
    medium['EX_glc_e'] = 3
    medium['EX_fru_e'] = 0
    print(medium)
    model.medium = medium
    solution= model.optimize()
    
    print('The current theoretical maximum biomass productivity:', solution.fluxes['EX_BIOMASS_c'], 'h')
    print('----------------')
    
    model.objective = model.reactions.EX_pbhb_e
    phb_production=model.optimize().objective_value
    print('The current maximum theoretical productivity for making PHB:', phb_production , 'mmol/gDW*h')
    print('----------------')
    print(model.medium)
    print(model.summary())
    #maximum_yield = phb_production / (-1*(model.reactions.EX_glc_e.flux))
    print(model.reactions.EX_glc_e.flux)
    #print('Maximum theoretical yield =', maximum_yield, 'mmol phb/mmol glucose')

{'EX_fe2_e': 10.0, 'EX_mg2_e': 10.0, 'EX_pi_e': 100.0, 'EX_cobalt2_e': 10.0, 'EX_cl_e': 10.0, 'EX_k_e': 10.0, 'EX_fe3_e': 10.0, 'EX_so4_e': 10.0, 'EX_fru_e': 0, 'EX_nh4_e': 10.0, 'EX_na_e': 10.0, 'EX_o2_e': 18.5, 'EX_mobd_e': 10.0, 'EX_h2o_e': 1000.0, 'EX_h_e': 100.0, 'EX_glc_e': 3}
The current theoretical maximum biomass productivity: 0.0 h
----------------
The current maximum theoretical productivity for making PHB: 0.0 mmol/gDW*h
----------------
{'EX_fe2_e': 10.0, 'EX_mg2_e': 10.0, 'EX_pi_e': 100.0, 'EX_cobalt2_e': 10.0, 'EX_cl_e': 10.0, 'EX_k_e': 10.0, 'EX_fe3_e': 10.0, 'EX_so4_e': 10.0, 'EX_nh4_e': 10.0, 'EX_na_e': 10.0, 'EX_o2_e': 18.5, 'EX_mobd_e': 10.0, 'EX_h2o_e': 1000.0, 'EX_h_e': 100.0, 'EX_glc_e': 3}
Objective
1.0 EX_pbhb_e = 0.0

Uptake
------
Empty DataFrame
Columns: [Metabolite, Reaction, Flux, C-Number, C-Flux]
Index: []

Secretion
---------
Empty DataFrame
Columns: [Metabolite, Reaction, Flux, C-Number, C-Flux]
Index: []

0.0


### From the ilittereature we know R.eutropha can grow anarobically. But it seems like our model can not:

In [41]:
with model:
    medium = model.medium
    medium['EX_o2_e'] = 0
    print(medium)
    model.medium=medium
    solution= model.optimize()
    
    print('The current theoretical maximum biomass productivity:', solution.fluxes['EX_BIOMASS_c'], 'h')
    print('----------------')
    
    model.objective = model.reactions.EX_pbhb_e
    phb_production=model.optimize().objective_value
    print('The current maximum theoretical productivity for making PHB:', phb_production , 'mmol/gDW*h')
    print('----------------')
    
    maximum_yield = phb_production / (-1*(model.reactions.EX_fru_e.flux))
    print('Maximum theoretical yield =', maximum_yield, 'mmol phb/mmol Fructose')
    print(model.summary())

{'EX_fe2_e': 10.0, 'EX_mg2_e': 10.0, 'EX_pi_e': 100.0, 'EX_cobalt2_e': 10.0, 'EX_cl_e': 10.0, 'EX_k_e': 10.0, 'EX_fe3_e': 10.0, 'EX_so4_e': 10.0, 'EX_fru_e': 3.0, 'EX_nh4_e': 10.0, 'EX_na_e': 10.0, 'EX_o2_e': 0, 'EX_mobd_e': 10.0, 'EX_h2o_e': 1000.0, 'EX_h_e': 100.0, 'EX_glc_e': 1000.0}
The current theoretical maximum biomass productivity: 0.0 h
----------------
The current maximum theoretical productivity for making PHB: 1.6363636363636358 mmol/gDW*h
----------------
Maximum theoretical yield = 0.5454545454545453 mmol phb/mmol Fructose
Objective
1.0 EX_pbhb_e = 1.6363636363636382

Uptake
------
Metabolite  Reaction   Flux  C-Number   C-Flux
     fru_e  EX_fru_e      3         6  100.00%
     h2o_e  EX_h2o_e 0.5455         0    0.00%

Secretion
---------
Metabolite      Reaction    Flux  C-Number  C-Flux
      ac_e       EX_ac_e -0.5455         2   9.52%
    etoh_e      EX_eth_e  -2.182         2  38.10%
     for_e  EX_formate_e      -6         1  52.38%
       h_e        EX_h_e  -6.54

In [42]:
model.reactions.get_by_id('EX_eth_e')

Reaction identifier,EX_eth_e
Name,EX_eth_e
Memory address,0x0176511e9e20
Stoichiometry,etoh_e --> Ethanol -->
GPR,
Lower bound,0.0
Upper bound,1000.0


### Our model contains also exchangereactions for succinate, fumarate and pyruvate. We are checking out the yealds for growing on these C-sourses. 


In [43]:
with model:
    medium = model.medium
    medium['EX_fru_e'] = 0
    medium['EX_succ_e'] = 3
    print(medium)
    model.medium=medium
    
    solution= model.optimize()
    print('The current theoretical maximum biomass productivity:', solution.fluxes['EX_BIOMASS_c'], 'h')
    print('----------------')
    a=  solution.fluxes['EX_BIOMASS_c']
    
    model.objective = model.reactions.EX_pbhb_e
    phb_production=model.optimize().objective_value
    print('The current maximum theoretical productivity for making PHB:', phb_production , 'mmol/gDW*h')
    print('----------------')
    
    maximum_yield = phb_production / (-1*(model.reactions.EX_succ_e.flux))
    print('Maximum theoretical yield =', maximum_yield, 'mmol phb/mmol Succinate')
    print(model.summary())

{'EX_fe2_e': 10.0, 'EX_mg2_e': 10.0, 'EX_pi_e': 100.0, 'EX_cobalt2_e': 10.0, 'EX_cl_e': 10.0, 'EX_k_e': 10.0, 'EX_fe3_e': 10.0, 'EX_so4_e': 10.0, 'EX_fru_e': 0, 'EX_nh4_e': 10.0, 'EX_na_e': 10.0, 'EX_o2_e': 18.5, 'EX_mobd_e': 10.0, 'EX_h2o_e': 1000.0, 'EX_h_e': 100.0, 'EX_glc_e': 1000.0, 'EX_succ_e': 3}
The current theoretical maximum biomass productivity: 0.0 h
----------------
The current maximum theoretical productivity for making PHB: 1.9363636363636239 mmol/gDW*h
----------------
Maximum theoretical yield = 0.6454545454545413 mmol phb/mmol Succinate
Objective
1.0 EX_pbhb_e = 1.9363636363636239

Uptake
------
Metabolite   Reaction  Flux  C-Number   C-Flux
       h_e     EX_h_e     6         0    0.00%
      o2_e    EX_o2_e 1.786         0    0.00%
    succ_e  EX_succ_e     3         4  100.00%

Secretion
---------
Metabolite   Reaction   Flux  C-Number   C-Flux
     co2_e   EX_co2_e -4.255         1  100.00%
     h2o_e   EX_h2o_e -3.191         0    0.00%
     phb_e  EX_pbhb_e -1.9

In [44]:
model.reactions.get_by_id('EX_succ_e')

Reaction identifier,EX_succ_e
Name,EX_succ_e
Memory address,0x0176511c2790
Stoichiometry,succ_e --> Succinate -->
GPR,
Lower bound,0.0
Upper bound,1000.0


The results form growing on fumarate:

In [45]:
with model:
    medium = model.medium
    medium['EX_fru_e'] = 0
    medium['EX_fum_e'] = 3
    print(medium)
    model.medium=medium
    
    solution= model.optimize()
    print('The current theoretical maximum biomass productivity:', solution.fluxes['EX_BIOMASS_c'], 'h')
    print('----------------')
    
    model.objective = model.reactions.EX_pbhb_e
    phb_production=model.optimize().objective_value
    print('The current maximum theoretical productivity for making PHB:', phb_production , 'mmol/gDW*h')
    print('----------------')
    
    maximum_yield = phb_production / (-1*(model.reactions.EX_fum_e.flux))
    print('Maximum theoretical yield =', maximum_yield, 'mmol phb/mmol Fumarate')


{'EX_fe2_e': 10.0, 'EX_mg2_e': 10.0, 'EX_pi_e': 100.0, 'EX_cobalt2_e': 10.0, 'EX_cl_e': 10.0, 'EX_k_e': 10.0, 'EX_fe3_e': 10.0, 'EX_so4_e': 10.0, 'EX_fru_e': 0, 'EX_nh4_e': 10.0, 'EX_na_e': 10.0, 'EX_o2_e': 18.5, 'EX_mobd_e': 10.0, 'EX_h2o_e': 1000.0, 'EX_h_e': 100.0, 'EX_glc_e': 1000.0, 'EX_fum_e': 3}
The current theoretical maximum biomass productivity: 0.0 h
----------------
The current maximum theoretical productivity for making PHB: 1.827272727272728 mmol/gDW*h
----------------
Maximum theoretical yield = 0.6090909090909093 mmol phb/mmol Fumarate


The results form growing on pyruvate:

In [76]:
with model:
    medium = model.medium
    medium['EX_fru_e'] = 0
    medium['EX_pyr_e'] = 3
    print(medium)
    model.medium=medium
    
    solution= model.optimize()
    print('The current theoretical maximum biomass productivity:', solution.fluxes['EX_BIOMASS_c'], 'h')
    print('----------------')
    
    model.objective = model.reactions.EX_pbhb_e
    phb_production=model.optimize().objective_value
    print('The current maximum theoretical productivity for making PHB:', phb_production , 'mmol/gDW*h')
    print('----------------')
    
    maximum_yield = phb_production / (-1*(model.reactions.EX_pyr_e.flux))
    print('Maximum theoretical yield =', maximum_yield, 'mmol phb/mmol Pyruvate')

{'EX_fe2_e': 10.0, 'EX_mg2_e': 10.0, 'EX_pi_e': 100.0, 'EX_cobalt2_e': 10.0, 'EX_cl_e': 10.0, 'EX_k_e': 10.0, 'EX_fe3_e': 10.0, 'EX_so4_e': 10.0, 'EX_fru_e': 0, 'EX_nh4_e': 10.0, 'EX_na_e': 10.0, 'EX_o2_e': 18.5, 'EX_mobd_e': 10.0, 'EX_h2o_e': 1000.0, 'EX_h_e': 100.0, 'EX_glc_e': 1000.0, 'EX_pyr_e': 3}
The current theoretical maximum biomass productivity: 0.11617084064312147 h
----------------
The current maximum theoretical productivity for making PHB: 1.6016949152542397 mmol/gDW*h
----------------
Maximum theoretical yield = 0.5338983050847466 mmol phb/mmol Succinate
{'EX_fe2_e': 10.0, 'EX_mg2_e': 10.0, 'EX_pi_e': 100.0, 'EX_cobalt2_e': 10.0, 'EX_pyr_e': 0, 'EX_cl_e': 10.0, 'EX_k_e': 10.0, 'EX_fe3_e': 10.0, 'EX_so4_e': 10.0, 'EX_nh4_e': 10.0, 'EX_na_e': 10.0, 'EX_o2_e': 18.5, 'EX_mobd_e': 10.0, 'EX_h2o_e': 1000.0, 'EX_h_e': 100.0, 'EX_glc_e': 1000.0, 'EX_fru_e': 3}


### With the data constructed in this file. Here is a table combinding the data: